In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV


#Tal ve me hubiera venido bien comenzar con todas las importaciones del proyecto
#anterior, pero creo que tampoco es cuestión de estar importando librerías sin saber
#siquiera si las voy a necesitar aquí o no

df = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews.csv")
df.head()

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0


In [11]:
#En este caso no toca hacer ningún EDA, ya que solamente nos dan la review, la app
#desde la que la escriben y la "polarity" que es 0 o 1 según sea mala o buena la review

#Lo primero es eliminar la variable de package_name
#Es posible que según cuál sea la aplicación usada haya más reseñas buenas o malas?
#Sí, pero hay que entender que lo que estamos buscando no es predecir si van a ser
#buenas o malas, sino evaluar si lo son. Para evaluar el contenido de un mensaje
#no es necesario saber desde que app se ha escrito

#Por otro lado tenemos que "Eliminar espacios y convertir a minúsculas el texto"
#tal y como indica el enunciado

#Puedo hacer una mini función que sea procesar_df

def procesar_df(df):
    df = df.drop("package_name", axis=1)
    df["review"] = df["review"].str.strip().str.lower()
    return df

df = procesar_df(df)


In [14]:
#Bien, ahora ya trabajamos solamente con review y polarity
#Hago la separación clásica dejando el 20% para test

X = df["review"]
y = df["polarity"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)

In [15]:
#Esta parte del código es parte del enunciado. Lo que estamos haciendo es simplemente
#convertir la información en una matriz con las frecuencias de cada una, entrenamos
#con el transformador y aplicamos el test

vec_model = CountVectorizer(stop_words="english")
X_train = vec_model.fit_transform(X_train).toarray()
X_test = vec_model.transform(X_test).toarray()

In [18]:
#Ahora el enunciado nos dice que razonemos qué modelo va a tener mejores resultados
#entre estos tres: GaussianNB, MultinomialNB o BernoulliNB

#Me quedo con el multinomial porque estamos jugando con frecuencias de palabras donde
#además el objetivo es binario

model = MultinomialNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [20]:
#A ver qué tal lo ha hecho
accuracy_score(y_test, y_pred)

#Un 73,18% no está mal, pero no podemos comprarlo con nada. Ahora toca entrenarlo y
#probarlo con los otros dos modelos para ver si estábamos en lo cierto al elegir multinomial

0.7318435754189944

In [24]:
model = GaussianNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.7039106145251397

In [25]:
model = BernoulliNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.6815642458100558

In [ ]:
#Pues sí, hemos superado a ambos de manera bastante clara. Siempre hay que tener presente
#que aquí estamos hablando de lenguaje natural y que este análisis está teniendo en cuenta
#solamente las frecuencias de las palabras (no su orden en una frase o los significados
#de las palabras, sino simplemente su correlación con uan review positiva o negativa)

#Por qué digo esto? En español podemos decir algo como "No está nada mal"
#Que tiene las mismas palabras que "Nada, no está. Mal!"
#Y ambas tienen significados opuestos
#Estoy orgulloso de este ejemplo

#Sé que estas reviews están en inglés, pero la ironía está presente en mayor o menor
#medida en todas las culturas e idiomas

#Es por esto que estos modelos son capaces de predecir si la review es buena o no sin
#hablar el idioma que están analizando.

In [32]:
#Ahora toca mejorar este modelo. Una buena forma es con un random forest

modelo_rf = RandomForestClassifier(random_state=57)
modelo_rf.fit(X_train, y_train)

#Hace sus predicciones y evalúa
predic_rf = modelo_rf.predict(X_test)
precis_rf = accuracy_score(y_test, predic_rf)

print(f"Random Forest Accuracy: {precis_rf}")

#Bastante mejor. He probado con varios random_state y el mejor ha sido 57
#Puede parecer que no hay una gran diferencia entre el 73% de antes y el 77% de ahora
#pero hayq ue tener en cuenta que acercarnos al 100%, además de imposible, es exponencialmente
#más difícil cuanto más cerca estamos, por lo que un 77% sí que es claramente superior a un 73%

Random Forest Accuracy: 0.770949720670391


In [44]:
#Sospecho que hay que optimizar más este modelo. Voy a hacer una busqueda aleatoria
#de hiperparámetros para el rf


rf_hiperparametros = {
    "n_estimators": [200, 300, 400, 500],
    "max_depth": [2, 3, 4, 5, 10, None],
    "min_samples_split": [2, 3, 5, 10]
}

rf_random_search = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_distributions=rf_hiperparametros,
    n_iter=40,
    scoring="accuracy",
    cv=3,
    random_state=42,
    n_jobs=-1
)

rf_random_search.fit(X_train, y_train)

print(f"Mejores hiperparámetros: {rf_random_search.best_params_}")

#Aquí reentreno con los mejores hiperparámetros
best_rf_simple = rf_random_search.best_estimator_
best_rf_simple.fit(X_train, y_train)

#Y sacamos la evaluación del modelo
rf_simple_pred = best_rf_simple.predict(X_test)
rf_simple_accuracy = accuracy_score(y_test, rf_simple_pred)
print(f"Random forest optimizado: {rf_simple_accuracy}")

#En este caso parece que no ha sido posible mejorar el random forest original. En cualquier
#caso, me quedo con el 77% de performance del rf y el 73% del naive_bayes como comparativa

Mejores hiperparámetros: {'n_estimators': 300, 'min_samples_split': 3, 'max_depth': None}
Random forest optimizado: 0.7430167597765364


In [39]:
#Esto era otro modelo que quedó en pruebas también mejores que GaussianNB y que BernoulliNB
#pero no mejores que MultinomialNB ni que el random forest

Mejor modelo encontrado: MultinomialNB(alpha=0.5, fit_prior=False)
Mejores hiperparámetros: {'alpha': 0.5, 'fit_prior': False}
Exactitud en el conjunto de prueba: 0.7150837988826816
